In [1]:
#Installing and Loading the required libraries 
!pip install -q -U bitsandbytes transformers peft accelerate datasets scipy

import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, Trainer, TrainingArguments
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, PeftModel # modules for QLoRA and PEFT
from datasets import load_dataset
import warnings
warnings.filterwarnings("ignore")

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2024.4.0 which is incompatible.
libpysal 4.9.2 requires packaging>=22, but you have packaging 21.3 which is incompatible.
libpysal 4.9.2 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
momepy 0.7.0 requires shapely>=2, but you have shapely 1.8.5.post1 which is incompatible.
spopt 0.6.0 requires shapely>=2.0.1, but you have shapely 1.8.5.post1 which is incompatible.
ydata-profiling 4.6.4 requires numpy<1.26,>=1.16.0, but you have numpy 1.26.4 which is incompatible.
ydata-profiling 4.6.4 requires scipy<1.12,>=1.4.1, but you have scipy 1.13.0 which is incompatible.


2024-04-16 19:30:24.610420: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-16 19:30:24.610519: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-16 19:30:24.728540: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
#Loading the pretrained model
#pre-trained language model to be loaded from Hugging Face's model hub
llm_model = "mistralai/Mixtral-8x7B-Instruct-v0.1"

#tokenizer corresponding to the model to convert text into a format that the model can understand
tokenizer = AutoTokenizer.from_pretrained(llm_model)

# Configure quantization settings using bitsandbytes library (QLoRa configarations)
# This setup is for loading the model with 4-bit weights using NF4 quantization
# NF4 is a specific quantization technique optimized for the model
# `load_in_4bit`: Enables loading models in 4-bit precision
# `bnb_4bit_quant_type`: Sets the type of 4-bit quantization (NF4 in this case)
# `bnb_4bit_use_double_quant`: Enables double quantization to further reduce memory usage
# `bnb_4bit_compute_dtype`: Sets the computation data type to bfloat16, balancing performance and precision
double_quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16
)

# Load the pre-trained causal language model with the specified quantization configuration
# `device_map="auto"`: Automatically assigns model layers to available GPUs, optimizing for performance
model = AutoModelForCausalLM.from_pretrained(
    llm_model,
    quantization_config=double_quant_config, 
    device_map="auto"
)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

...

model-00017-of-00019.safetensors:   0%|          | 0.00/4.90G [00:00<?, ?B/s]

model-00018-of-00019.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [3]:
#Preparing the Model for QLoRA
# Prepare the model for training with quantization (such as k-bit quantization) to enhance efficiency
model = prepare_model_for_kbit_training(model)

# Defined constants for the LoRA configuration:
CUTOFF_LEN = 256  #The maximum sequence length for processing
LORA_R = 8 # The rank of the low-rank matrices, controlling the amount of parameters to train
LORA_ALPHA = 2 * LORA_R # scaling factor for the low-rank matrices, calculated as twice the rank
LORA_DROPOUT = 0.1 #Dropout rate to prevent overfitting in the added LoRA layers

# Configure the LoRA settings specific to the model’s needs:
# `r`: Rank of the adaptation matrix
# `lora_alpha`: Scaling factor for adaptation
# `target_modules`: Specifies which layers (MoE layers here) are targeted for low-rank parameterization
# `lora_dropout`: Dropout rate applied to LoRA parameters
# `bias`: Specifies if biases should be adapted; 'none' indicates no bias adaptation
# `task_type`: Defines the type of task the model is being prepared for, here it is causal language modeling

config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=[ "w1"],  #just targetting the MoE layers.
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

# Apply the PEFT model configuration to the prepared model
model = get_peft_model(model, config)

In [4]:
# the number of trainable parameters, total parameters, and the percentage of parameters that are trainable in a model
def print_trainable_parameters(m):
    trainable_params = sum(p.numel() for p in m.parameters() if p.requires_grad)
    all_params = sum(p.numel() for p in m.parameters())
    print(f"trainable params: {trainable_params} || all params: {all_params} || trainable%: {100 * trainable_params / all_params}")

print_trainable_parameters(model)

trainable params: 37748736 || all params: 23520350208 || trainable%: 0.16049393680864704


Mixtral 8x7B outperforms Llama 2 70B on most benchmarks with 6x faster inference. It is the strongest open-weight model with a permissive license and the best model overall regarding cost/performance trade-offs. In particular, it matches or outperforms GPT3.5 on most standard benchmarks.

Mixtral-8x7B is the second large language model (LLM) released by mistral.ai, after Mistral-7B.

In [5]:
# loading data from local directory in csv format
data = load_dataset('csv', data_files='/kaggle/input/d/alexwanjai/contradictions/Contradiction.csv')

# loading data from hugging face datasets
#data = load_dataset("harpreetsahota/modern-to-shakesperean-translation")
# extracting traning data
train_data = data['train']
# train_data view
train_data[10]

Generating train split: 0 examples [00:00, ? examples/s]

{'Type': 'Implicit Contradiction:',
 'Contradiction': ' "The government claims to prioritize environmental protection.",  "Yet, they continue to approve projects that harm ecosystems."'}

In [6]:
# use a common character as a padding token, '!' is used temporarily as a placeholder
tokenizer.pad_token = "!" #Not EOS, will explain another time.

#Loading and Preparing the Dataset
#a function to generate a text prompt for translation tasks
def generate_prompt(user_query,  sep="\n\n### "):  
    # a system message that instructs what the task is
    sys_msg= "Check if tere is any contrudiction in the depostion and highlight the type of contrudiction."
    # Create the prompt by combining the instruction with the user's modern text input and the corresponding Shakespearean translation
    # This format helps in guiding the model to perform the translation task
    p =  " [INST]" + sys_msg +"\n"+ user_query["Contradiction"] + "[/INST]" +  user_query["Type"] + ""
    return p

# a function to tokenize the generated prompt
def tokenize(prompt):
    # Tokenize the prompt and append the End Of String token
    # Apply truncation to limit the prompts to a maximum length and pad any shorter prompts to this length
    return tokenizer(
        prompt + tokenizer.eos_token, # Add the end-of-sequence token to each prompt to signal the end of text
        truncation=True, # Ensure that prompts longer than the max length are truncated to fit
        max_length=CUTOFF_LEN ,# Set the maximum prompt length (CUTOFF_LEN must be defined elsewhere in your script)
        padding="max_length" # Pad shorter prompts to ensure consistent length for batch processing
    )

# Shuffle the training data and apply the tokenize function to each example
# Remove the original columns 'modern' and 'shakespearean' to prevent leakage and reduce memory usage
dataset = train_data.shuffle().map(lambda x: tokenize(generate_prompt(x)),remove_columns=["Contradiction" , "Type"])

# Output the processed dataset
dataset

Map:   0%|          | 0/54 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 54
})

In [7]:
#dataset[1]

In [8]:
#Setting Up the Training Arguments
training_args = TrainingArguments(
    per_device_train_batch_size=1,  # Number of samples to process before moving to the next batch, per device
    gradient_accumulation_steps=4,  # Number of samples to process before moving to the next batch, per device
    num_train_epochs=6,  # Total number of training epochs (full passes over the training dataset)
    learning_rate=1e-4, # Learning rate for the optimizer
    logging_steps=2, # Frequency of logging predictions (every 2 steps)
    optim="adamw_torch", # Specify optimizer type, here it's AdamW as implemented in PyTorch
    save_strategy="epoch", # Strategy to save the model checkpoint ('epoch' to save at the end of each epoch)
    weight_decay=0.01, # Strength of weight decay
    output_dir="mixtral-contradiction_detector" # Directory where the training outputs (models, logs) will be saved
)

# These arguments will guide how the model is trained, including how often it updates the weights,
# how it handles batches of data, and where it saves its outputs.

In [9]:
#Training the Model
trainer = Trainer( # The Trainer handles the training and evaluation of the model.
    model=model,# The pre-trained model that you want to fine-tune.
    train_dataset=dataset,# The dataset to use for training, prepared earlier.
    args=training_args,# Training arguments set previously, defining the training parameters.
    data_collator=transformers.DataCollatorForLanguageModeling(
        tokenizer, # The tokenizer used for pre-processing text data.
        mlm=False # Specifies that this is not a Masked Language Model (MLM) training.
    ),
)

# Disable caching to potentially reduce memory usage during traininga and
# speed up training by storing past computations but uses more memory.
model.config.use_cache = False

In [10]:
# Training
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
2,7.074000
4,5.757100
6,4.759900
8,4.090500
10,3.364700
12,2.770000
14,2.226100
16,1.928900
18,1.687200
20,1.525800


TrainOutput(global_step=78, training_loss=1.4668368849998865, metrics={'train_runtime': 6863.5174, 'train_samples_per_second': 0.047, 'train_steps_per_second': 0.011, 'total_flos': 2.233674925867008e+16, 'train_loss': 1.4668368849998865, 'epoch': 5.78})